## Exercise 2.73
**Section 2.3.2 described a program that performs symbolic differentiation:**
```scheme
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp)
         (if (same-variable? exp var) 1 0))
        ((sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var)))
        ((product? exp)
         (make-sum (make-product
                    (multiplier exp)
                    (deriv (multiplicand exp) var))
                   (make-product
                    (deriv (multiplier exp) var)
                    (multiplicand exp))))
        <more rules can be added here>
        (else (error "unknown expression type: DERIV" exp))))
```
**We can regard this program as performing a dispatch on
the type of the expression to be differentiated. In this situation the "type
tag" of the datum is the algebraic operator symbol (such as +) and the operation being performed is
`deriv`. We can transform this program into data-directed style by rewriting the basic derivative
 procedure as**

```scheme
(define (deriv exp var)
 (cond ((number? exp) 0)
  ((variable? exp) (if (same-variable? exp var) 1 0)) 
  (else ((get 'deriv (operator exp))
         (operands exp) var))))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))
```

a. **Explain what was done above. Why can’t we assimilate the predicates
   `number?` and `variable?` into the data-directed dispatch?**
   
b. **Write the procedures for derivatives of sums and products,
   and the auxiliary code required to install them in the table used by the program above.**
   
c. **Choose any additional differentiation rule that you like,
   such as the one for exponents (Exercise 2.56), and install it in this data-directed system.**
   
d. **In this simple algebraic manipulator the type of an expression is the algebraic operator that binds it
   together. Suppose, however, we indexed the procedures in the opposite way, so that the dispatch line in `deriv`
   looked like**
   ```
   ((get (operator exp) 'deriv) (operands exp) var)
   ```
   **What corresponding changes to the derivative system are required?**

a. Above, instead of relying on our pre-defined predicates `sum?` and `product?`, the `deriv` function was changed to invoke the correct differentiation logic using the table-based dispatch that was described in the text. Assuming the table is correctly populated, upon calling the new `deriv`, a lookup is performed on the table to get the correct procedure to apply for the operation `deriv` and the given type of expression, which in our case (confusingly enough) is equivalent to the operator of the algebraic expression. The exception to this are of course inputs that are numbers or variables, because in our system we represent them as raw values (like `42` or `'z`) instead of as data with type tags.

b. The procedures are:

```scheme
; (We assume here that the arguments to deriv-sum and
; deriv-prod are passed as a pair containing the operands)

; Derivative of sum
(define (deriv-sum sum-pair var)
  (make-sum (deriv (car sum-pair) var)
            (deriv (cdr sum-pair) var)))

; Derivative of product
(define (deriv-prod prod-pair var)
  (make-sum (make-product
             (car prod-pair)
             (deriv (cdr prod-pair) var))
            (make-product
             (deriv (car prod-pair) var)
             (cdr prod-pair))))

; Auxiliary code to install them in the table
(put 'deriv ('+) deriv-sum)
(put 'deriv ('*) deriv-sum)
```

c. Below is the function to differentiate exponents (which will only work in cases where the exponent is a constant!) and a function call to install it.
```scheme
; Here, for simplicity, I will say that exp-pair can be any structure
; containing the base and the exponent parts, as long as (base exp-pair)
; and (exponent exp-pair) return those parts.
(define (deriv-exp exp-pair var)
  (make-product (make-product (exponent exp-pair)
                              (make-exponentiation (base exp-pair) (make-sum (exponent exp-pair) -1)))
                (deriv (base exp-pair) var)))

; Now, the following call should install it in the table
(put 'deriv ('**) deriv-exp)
```

d. Since the implementations themselves only take the operands they need and the variable with respect to which we are differentiating, and the logic of storing and fetching the appropriate function is external to them, no changes are necessary.

## Exercise 2.74
**Insatiable Enterprises, Inc., is a highly de-centralized conglomerate company consisting of a large number of independent divisions located all over the world. The company’s computer facilities have just been interconnected by means of a clever network-interfacing scheme that makes the entire network appear to any user to be a single computer. Insatiable’s president, in her ﬁrst attempt to exploit the ability of the ne
twork to extract administrative information from division files, is dismayed to discover that, although all the division files have been implemented as data structures in Scheme, the particular data structure used varies from division to division. A meeting of division managers is hastily called to search for a strategy to integrate the files that will satisfy headquarters’ needs while preserving the existing autonomy of the divisions. Show how such a strategy can be implemented with data-directed programming. As an example, suppose that each division’s personnel records consist of a single file, which contains a set of records keyed on employees’ names. The structure of the set varies from division to division. Furthermore, each employee’s record is itself a set (structured differently from division to division) that contains information keyed under identifiers such as address and salary.  In particular:**

**a. Implement for headquarters a `get-record` procedure that retrieves a specified employee’s record from a specified personnel file. The procedure should be applicable to any division’s file. Explain how the individual divisions’ files should be structured. In particular, what type information must be supplied**?

**b. Implement for headquarters a `get-salary` procedure that returns the salary information from a given employee’s record from any division’s personnel file. How should the record be structured in order to make this operation work**?

**c. Implement for headquarters a `find-employee-record procedure`. This should search all the divisions’ files for the record of a given employee and return the record. Assume that this procedure takes as arguments an employee’s name and a list of all the divisions’ files.**

**d. When Insatiable takes over a new company, what changes must be made in order to incorporate the new personnel information into the central system?**


a. The individual divisions' files can be structured in any way they like, provided that they do the following things:
1. Have an implementation of `get-record` that takes the employee name and produces the record.
2. The file should be defined as a function which returns a function that takes an abstract operation name like `'get-record` and returns the implementation of that operation for that file's specific data structure. Now, the generic `get-record` procedure can be implemented like this:
```scheme
(define (get-record name file)
  ((file 'get-record) name))
```

b. Similar to the above, the division's files should be structured in a way that `(file 'get-salary)` evaluates to a function that takes the employee's name and returns the salary if a record is found.
```scheme
(define (get-salary name file)
  ((file 'get-salary) name))
```

c. For this to work, the `get-record` procedure mentioned in question a. (the divisions' implementations) should return false if no employee with that name exists. Then, we can call `get-record` on each file until an employee is found.
```scheme
(define (find-employee-record name files)
  (if (null? files)
      #f
      (if (get-record name (car files))
          (get-record name (car files))
          (find-employee-record name (cdr files)))))
```
The above procedure will return`#f`is no such employee record is found.

d. The only changes required to incorporate the new personnel information into the central system are:
1. Implementing `get-record`, `get-salary` and other accessors in the new company's file.
2. Including the new company's file in the central list of files.

## Exercise 2.75
**Implement the constructor `make-from-mag-ang` in message-passing style. This procedure should be analogous to the `make-from-real-imag` procedure given above.**

In [8]:
(define (make-from-mag-ang mag ang)
  (define (dispatch op)
    (cond ((eq? op 'magnitude) mag)
          ((eq? op 'angle) ang)
          ((eq? op 'real-part) (* mag (cos ang)))
          ((eq? op 'imag-part) (* mag (sin ang)))
          (else (error "Unknown op: MAKE-FROM-REAL-IMAG" op))))
  dispatch)

(define a (make-from-mag-ang 2 1))
(display (a 'real-part))
(newline)
(display (a 'imag-part))

1.0806046117362795
1.682941969615793

## Exercise 2.76
**As a large system with generic operations evolves, new types of data objects or new operations may be needed. For each of the three strategies—generic operations with explicit dispatch, data-directed style, and message-passing-style—describe the changes that must be made to a system in order to add new types or new operations. Which organization would be most appropriate for a system in which new types must often be added? Which would be
most appropriate for a system in which new operations must often be added?**

With explicit dispatch, each time a new type is added, all of the existing generic operations must be modified to detect the type of the datum and perform dispatch to the specific operation. Conversely, when adding a new operation, the operation must be implemented for all the data types, and a generic operation created for it.

In the data-directed style, when a new type is added, the new data type package simply needs to define its operations internally and install its operations into the table. No modification of the general operation is necessary. When a new operation is added, it is still necessary to modify each package to support that operation.

With the message passing style, similar to the data-directed style, adding a data type just means implementing a package with the operations for that specific type. However, in contrast to the data-directed style, it is not necessary to install operations on a table, because the dispatch is performed internally by the data object itself. When adding a new operation, the operation must be implemented by each package.

For a system in which new types must often be added, either the message passing style or the data-directed style is appropriate, as the changes required are additive. If new operations must often be added, then one approach is not significantly better than any other, because the specific operation must be implemented for each type in any case.